# Updated Model

Notebook by: JIEYAO WANG


Coding up the model under the new setting; 

#### import packages

In [1]:
# Required packages
import os
import sys
sys.path.append('./src')
from supportfunctions import *
sys.stdout.flush()

#### for now set the damage function to be the low damage case only

In [2]:
# Damage function choices
damageSpec = 'Low'  # Choose among "High"(Weitzman), 'Low'(Nordhaus) and 'Weighted' (arithmeticAverage of the two)

if damageSpec == 'High':
    weight = 0.0
elif damageSpec == 'Low':
    weight = 1.0
else:
    weight = 0.5

xi_a =  0.1  # Ambiguity Averse Paramter 

if xi_a < 1:
    aversespec = "Averse"
else:
    aversespec = 'Neutral'

smart_guess = False

### parameter set up-add in new parameter for temperature model

In [3]:
McD = np.loadtxt('./data/TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

beta_f = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
sigma_f = np.var(par_lambda_McD, ddof = 1)  # varaiance of climate sensitivity parameters
lambda_f = 1.0 / sigma_f 

# define global variables
delta = 0.01        
kappa = 0.032       
sigma_k = 0.0161
sigma_r = 0.0339 
alpha = 0.115000000000000
phi_0 = 0.0600
phi_1 = 16.666666666666668
mu_k = -0.034977443912449
psi_0 = 0.112733407891680
psi_1 = 0.142857142857143

# parameters for damage function settings
power = 2 
gamma_1 = 0.00017675
gamma_2 = 2. * 0.0022
gamma_2_plus = 2. * 0.0197
bar_gamma_2_plus = weight * 0 + (1 - weight) * gamma_2_plus

sigma_1 = 0
sigma_2 = 0
rho_12 = 0
F_bar = 2
crit = 2
F0 = 1

xi_d = -1 * (1 - kappa)
xi_k = -1 * (1 - kappa)

epsilon = 0.3
tol = 1e-8

### arbitrary assumption on kappa_ZG
kappa_mean = 1
kappa_var = 0.1
c_temp = 3.154e8 # which unit to use? # 31.54
k_temp = 5.35 # Note BBH_ZG_WriteUp_v5b.pdf
### # Note BBH_ZG_WriteUp_v5b.pdf
sigma_T = 0.3 # or sqrt(0.3)
### arbitrary assumption on gamma_bar
gamma_bar = 290
### use F_0 to be the F_0 in this model as well
F_0 = 870 - 580
Q_0 = 342.5 # Note BBH_ZG_WriteUp_v5b.pdf
# Q_0 = 342.5-.5
# Q_0 = 342.5+.5


### state space set up-adjust K to T

In [4]:
R_min = 0
R_max = 9
F_min = 100
F_max = 4000
T_min = 200
T_max = 350

hR = 0.5
hF = 100
hT = 20

R = np.arange(R_min, R_max + hR, hR)
nR = len(R)
F = np.arange(F_min, F_max + hF, hF)
nF = len(F)
T = np.arange(T_min, T_max + hT, hT)
nT = len(T)

(R_mat, F_mat, T_mat) = np.meshgrid(R,F,T, indexing = 'ij')
stateSpace = np.hstack([R_mat.reshape(-1,1,order = 'F'),F_mat.reshape(-1,1,order = 'F'),T_mat.reshape(-1,1,order = 'F')])




### other numerical assumptions

In [5]:
quadrature = 'legendre'
n = 30
a = kappa_mean - 5 * np.sqrt(kappa_var)
b = kappa_mean + 5 * np.sqrt(kappa_var)

FC_Err = 1
episode = 0


### Enclosed functions

In [6]:
def initialize():
    v0 = kappa * R_mat - sigma_f * F_mat
    return v0

def update_e():
    global v0
    V_r = finiteDiff(v0,0,1,hR)
    V_r[V_r < 1e-16] = 1e-16
    V_f = finiteDiff(v0,1,1,hF)
    e = (delta * kappa) / ((V_r - V_f) * np.exp(R_mat) )
    return e

def g_1_func():
    c_1 = .15
    c_2 = .7
    c = c_temp
    # T_0 = (1.9e-15) ** (-1/6)
    T_0 = 288.05
    T_c = 273
    sigma_ZG = 5.6697e-8
    m = .4
    mu = 1
    g1 = (mu * Q_0 * (1 - c_1 - c_2/2) - sigma_ZG * (T_mat) ** 4 * (1 - m * np.tanh((T_mat / T_0) ** 6))) / c
    return g1


def g_2_func(kappa_ZG = 1):
    c_1 = .15
    c_2 = .7
    c = c_temp
# T_0 = (1.9e-15) ** (-1/6)
    T_0 = 288.05
    T_c = 273
    sigma_ZG = 5.6697e-8
    m = .4
    mu = 1
    g2 = (mu * Q_0 * (c_2/2) * np.tanh(kappa_ZG * (T_mat - T_c))) / c
    return g2

def prior_kappa(kappa_ZG):  
    # assumption??
    return kappa_ZG


def num_1(kappa_ZG = 1):
    global v0
    v0_dt = finiteDiff(v0,2,1,hT)
    num1 = np.exp((-1/xi_a) * (xi_d * (gamma_1 + gamma_2 * T_mat) + v0_dt) * (g_2_func(kappa_ZG)))
    return num1

def scale_1_func(x):
    return num_1(x) * norm.pdf(x,kappa_mean,np.sqrt(kappa_var))
  
def I_1():
    scale_1 = quad_int(scale_1_func, a, b, n, 'legendre')
    return -xi_a * np.log(scale_1)

def J_1():
    v0_dt = finiteDiff(v0,2,1,hT)
    scale_1 = quad_int(scale_1_func, a, b, n, 'legendre')
    def func_combine(x):
        return g_2_func(x) * scale_1_func(x)
    J1 = ((xi_d * (gamma_1 + gamma_2 * T_mat) + v0_dt) \
           * quad_int(func_combine, a, b, n, 'legendre')) / (scale_1)
    return J1

def R_1():
    return 1/xi_a * (I_1() - J_1())

def q_1(kappa_ZG):
    scale_1 = quad_int(scale_1_fnc, a, b, n, 'legendre')
    return num_1(kappa_ZG) / scale_1

def num_2(kappa_ZG):
    global v0
    v0_dt = finiteDiff(v0,2,1,hT)
    first_term = xi_d * (gamma_2_plus * (T_mat - gamma_bar) *(T_mat - gamma_bar) >= 0) * (g_1_func() + g_2_func(kappa_ZG))
    second_term = (xi_d * (gamma_1 + gamma_2 * T_mat) + v0_dt) * (g_2_func(kappa_ZG))
    third_term = xi_d * (gamma_2_plus * (T_mat - gamma_bar) * ((T_mat - gamma_bar)>= 0) * (k_temp / c_temp) * np.log(F_mat / F_0))
    num2 = np.exp(-1/xi_a * (first_term + second_term + third_term))
    return num2

def scale_2_func(x):
    return num_2(x) * norm.pdf(x,kappa_mean,np.sqrt(kappa_var))

def q_2(kappa_ZG):
    scale_2 = quad_int(scale_2_func, a, b, n, 'legendre')
    return num_2(kappa_ZG) / scale_2

def I_2():
    scale_2 = quad_int(scale_2_func, a, b, n, 'legendre')
    return -xi_a * np.log(scale_2)

def J_2():
    scale_2 = quad_int(scale_2_func, a, b, n, 'legendre')
    v0_dt = finiteDiff(v0,2,1,hT)
    def first_term(x):
        return xi_d * (gamma_2_plus * (T_mat - gamma_bar) *(T_mat -gamma_bar) >= 0) * (g_1_func() + g_2_func(x))
    def second_term(x):
        return (xi_d * (gamma_1 + gamma_2 * T_mat) + v0_dt) * (g_2_func(x))
    third_term = xi_d * (gamma_2_plus * (T_mat - gamma_bar) * ((T_mat - gamma_bar)>= 0) * (k_temp / c_temp) * np.log(F_mat / F_0))
    def conbined_func(x):
        return first_term(x) + second_term(x) + third_term
    J2 = quad_int(conbined_func, a, b, n, 'legendre') / (scale_2)
    return J2

def R_2():
    return 1/xi_a * (I_2() - J_2())

def pi_tilde_1_norm():
    pi_tilde_1 = weight * np.exp(-1 / xi_a * I_1())
    pi_tilde_2 = (1 - weight) * np.exp(-1 / xi_a * I_2())
    pi_tilde_1_normed = (pi_tilde_1) / (pi_tilde_1 + pi_tilde_2)
    
    return pi_tilde_1_normed

def RelativeEntropy():
    if weight == 0 or weight == 1:
        RE = pi_tilde_1_norm() * R_1() + (1 - pi_tilde_1_norm()) * R_2()
    else:
        RE = pi_tilde_1_norm() * R_1() + (1 - pi_tilde_1_norm()) * R_2() \
        + pi_tilde_1_norm() * np.log(pi_tilde_1_norm() / weight) \
        + (1 - pi_tilde_1_norm()) * np.log((1 - pi_tilde_1_norm()) / (1 - weight))
    return RE

def I_term(): # not being used
    pi_tilde_1 = weight * np.exp(-1 / xi_a * I_1())
    pi_tilde_2 = (1 - weight) * np.exp(-1 / xi_a * I_2())
    return -1 * xi_a * np.log(pi_tilde_1 + pi_tilde_2)
    
def V_d_Drift_const():
    global v0
    v0_dt = finiteDiff(v0,2,1,hT)
    flow = (xi_d * (gamma_1 + gamma_2 * T_mat) * (k_temp/c_temp) * np.log(F_mat / F_0)) \
    + xi_d * (gamma_1 + gamma_2 * T_mat) * g_1_func()
    return flow

def V_t_Drift_const():
    return g_1_func() + (k_temp/c_temp) * np.log(F_mat / F_0)

def Misspec_term():
    return 0

def flow_util():
    global v0
    flow = delta * kappa * (np.log(update_e() + R_mat)) + Misspec_term() + V_d_Drift_const()
    return flow 


In [ ]:
start_time = time.time()
episode = 0

while episode == 0 or FC_Err > tol: 
    if episode ==0:
        v0 = initialize()
    else:
        vold = v0.copy()
        
        
    if episode > 2000:
        epsilon = 0.1
    elif episode > 1000:
        epsilon = 0.2
    else:
        pass
    
    
    e = update_e()

    drift_distort = (pi_tilde_1_norm() * J_1() + (1 - pi_tilde_1_norm()) * J_2())

    RE_total = xi_a * RelativeEntropy()

    A = -delta * np.ones(R_mat.shape)
    B_r = -e * np.exp(R_mat) - 0.5 * (sigma_r ** 2)
    B_f = e * np.exp(R_mat)
    B_t = V_t_Drift_const()
    C_rr = 0.5 * sigma_r ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    C_tt = 0.5 * (sigma_T/c_temp) ** 2 * np.ones(R_mat.shape)
    D =  flow_util() + V_t_Drift_const() + RE_total

    out = PDESolver(stateSpace, A, B_r, B_f, B_t, C_rr, C_ff, C_tt, D, v0, epsilon, solverType = 'False Transient')

    out_comp = out[2].reshape(v0.shape,order = "F")
    
    # Calculating PDE error and False Transient error
    
    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dt = finiteDiff(v0,2,1,hT)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dtt = finiteDiff(v0,2,2,hT)
    
    PDE_rhs = A * v0 + B_r * v0_dr + B_f * v0_df + B_t * v0_dt + C_rr * v0_drr + C_tt * v0_dtt + C_ff * v0_dff + D
    PDE_Err = np.max(abs(PDE_rhs))
    FC_Err = np.max(abs((out_comp - v0)))
    if episode % 10 == 0:
        print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
    episode += 1
    
    # step 9: keep iterating until convergence
    v0 = out_comp

print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
print("--- %s seconds ---" % (time.time() - start_time))



Episode 0: PDE Error: 0.0025149752; False Transient Error: 0.0007519680; Iterations: 4; CG Error: 0.0000000000
Episode 10: PDE Error: 0.0024403157; False Transient Error: 0.0007296385; Iterations: 4; CG Error: 0.0000000000
Episode 20: PDE Error: 0.0023678731; False Transient Error: 0.0007079721; Iterations: 4; CG Error: 0.0000000000
Episode 30: PDE Error: 0.0022975815; False Transient Error: 0.0006869491; Iterations: 4; CG Error: 0.0000000000
Episode 40: PDE Error: 0.0022293771; False Transient Error: 0.0006665503; Iterations: 4; CG Error: 0.0000000000
Episode 50: PDE Error: 0.0021631978; False Transient Error: 0.0006467574; Iterations: 4; CG Error: 0.0000000000
Episode 60: PDE Error: 0.0020989836; False Transient Error: 0.0006275522; Iterations: 4; CG Error: 0.0000000000
Episode 70: PDE Error: 0.0020366762; False Transient Error: 0.0006089173; Iterations: 4; CG Error: 0.0000000000
Episode 80: PDE Error: 0.0019762188; False Transient Error: 0.0005908358; Iterations: 4; CG Error: 0.0000

In [ ]:
v = np.array([12, 24, 36])   
w = np.array([45, 55])  

print(np.reshape(v, (3, 1)) * w) 

X = np.array([[12, 22, 33], [45, 55, 66]]) 

print(X + v)

print((X.T + w).T) 
